In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!/usr/bin/env python3
"""
Dataset Diagnostics - Understand Your Data Before Training
===========================================================
This script analyzes your preprocessed stamps to:
1. Verify normalization is correct
2. Check for data quality issues
3. Visualize sample stamps
4. Analyze class balance and sources
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm

# Config
DATA_DIR = Path("/kaggle/input/stamps-clean/stamps_clean")
METADATA_CSV = DATA_DIR / "metadata.csv"
WORK_DIR = Path("/kaggle/working")
N_SAMPLES = 10  # samples to visualize per class

def analyze_normalization(metadata, data_dir, n_check=100):
    """
    Check if data is properly normalized.
    Should see mean~0, std~1, values in [-5, 5] range.
    """
    print("\n" + "="*60)
    print("NORMALIZATION CHECK")
    print("="*60)
    
    # Sample random stamps
    sample_paths = metadata.sample(min(n_check, len(metadata)))['stamp_path'].tolist()
    
    all_values = []
    for path in tqdm(sample_paths, desc="Loading samples"):
        arr = np.load(data_dir / path)['x']
        all_values.append(arr.flatten())
    
    all_values = np.concatenate(all_values)
    
    print(f"\nStatistics across {n_check} random stamps:")
    print(f"  Mean:   {all_values.mean():.4f} (expect: ~0)")
    print(f"  Std:    {all_values.std():.4f} (expect: ~1-2)")
    print(f"  Min:    {all_values.min():.4f} (expect: >=-5)")
    print(f"  Max:    {all_values.max():.4f} (expect: <=5)")
    print(f"  Median: {np.median(all_values):.4f}")
    
    # Check for NaN/Inf
    n_nan = np.isnan(all_values).sum()
    n_inf = np.isinf(all_values).sum()
    print(f"\n  NaN count: {n_nan}")
    print(f"  Inf count: {n_inf}")
    
    if n_nan > 0 or n_inf > 0:
        print("  ⚠️  WARNING: Found NaN or Inf values!")
    
    # Distribution plot
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(all_values, bins=100, alpha=0.7, edgecolor='black')
    plt.xlabel('Pixel Value')
    plt.ylabel('Count')
    plt.title('Pixel Value Distribution (All Channels)')
    plt.axvline(x=0, color='r', linestyle='--', label='Mean=0')
    plt.axvline(x=-5, color='orange', linestyle='--', label='Clip bounds')
    plt.axvline(x=5, color='orange', linestyle='--')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.hist(all_values, bins=100, alpha=0.7, edgecolor='black', cumulative=True, density=True)
    plt.xlabel('Pixel Value')
    plt.ylabel('Cumulative Probability')
    plt.title('Cumulative Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(WORK_DIR / 'normalization_check.png', dpi=150)
    print(f"\n✓ Saved normalization plot: {WORK_DIR / 'normalization_check.png'}")
    plt.close()


def analyze_class_balance(metadata):
    """Analyze class distribution across splits and sources"""
    print("\n" + "="*60)
    print("CLASS BALANCE ANALYSIS")
    print("="*60)
    
    # Overall balance
    print("\n--- Overall ---")
    print(metadata.groupby('label').size())
    total_pos = (metadata['label'] == 1).sum()
    total_neg = (metadata['label'] == 0).sum()
    print(f"Imbalance ratio: 1:{total_neg/total_pos:.1f}")
    
    # By split
    print("\n--- By Split ---")
    split_balance = metadata.groupby(['split', 'label']).size().unstack(fill_value=0)
    print(split_balance)
    
    for split in ['train', 'val', 'test']:
        pos = split_balance.loc[split, 1] if 1 in split_balance.columns else 0
        neg = split_balance.loc[split, 0] if 0 in split_balance.columns else 0
        if pos > 0:
            print(f"  {split}: 1:{neg/pos:.1f}")
    
    # By source (negatives only)
    print("\n--- Negative Sources ---")
    neg_sources = metadata[metadata['label'] == 0].groupby('source').size()
    print(neg_sources)
    print(f"\nTotal negative sources: {len(neg_sources)}")
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Split balance
    split_balance.plot(kind='bar', ax=axes[0], color=['blue', 'red'])
    axes[0].set_title('Class Balance by Split')
    axes[0].set_xlabel('Split')
    axes[0].set_ylabel('Count')
    axes[0].legend(['Negative', 'Positive'])
    axes[0].grid(True, alpha=0.3)
    
    # Source distribution (negatives)
    neg_sources.plot(kind='bar', ax=axes[1], color='steelblue')
    axes[1].set_title('Negative Sample Sources')
    axes[1].set_xlabel('Source Type')
    axes[1].set_ylabel('Count')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    # Pie chart
    axes[2].pie([total_neg, total_pos], labels=['Negative', 'Positive'], 
                autopct='%1.1f%%', colors=['blue', 'red'], startangle=90)
    axes[2].set_title('Overall Class Distribution')
    
    plt.tight_layout()
    plt.savefig(WORK_DIR / 'class_balance.png', dpi=150)
    print(f"\n✓ Saved class balance plot: {WORK_DIR / 'class_balance.png'}")
    plt.close()


def visualize_samples(metadata, data_dir, n_samples=10):
    """Visualize sample stamps for each class"""
    print("\n" + "="*60)
    print("VISUALIZING SAMPLE STAMPS")
    print("="*60)
    
    for label in [0, 1]:
        label_name = "POSITIVE (Asteroid)" if label == 1 else "NEGATIVE (Non-asteroid)"
        samples = metadata[metadata['label'] == label].sample(min(n_samples, (metadata['label'] == label).sum()))
        
        fig, axes = plt.subplots(n_samples, 5, figsize=(15, 3*n_samples))
        if n_samples == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle(f'{label_name} Samples', fontsize=16, y=0.995)
        
        for i, (idx, row) in enumerate(samples.iterrows()):
            arr = np.load(data_dir / row['stamp_path'])['x']
            
            # Handle shape
            if arr.ndim == 3 and arr.shape[-1] == 5:
                arr = np.transpose(arr, (2, 0, 1))
            
            # Display each channel
            channel_names = ['F1', 'F2', 'F3', 'S2', 'S3']
            for j in range(5):
                ax = axes[i, j]
                im = ax.imshow(arr[j], cmap='gray', vmin=-3, vmax=3)
                ax.set_title(f"{channel_names[j]}\n{row['source']}", fontsize=8)
                ax.axis('off')
                
                if j == 4:  # Add colorbar to last channel
                    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        
        plt.tight_layout()
        save_name = f'samples_{"positive" if label == 1 else "negative"}.png'
        plt.savefig(WORK_DIR / save_name, dpi=150, bbox_inches='tight')
        print(f"✓ Saved {label_name} samples: {WORK_DIR / save_name}")
        plt.close()


def analyze_spatial_distribution(metadata):
    """Analyze spatial distribution of samples across folders"""
    print("\n" + "="*60)
    print("SPATIAL DISTRIBUTION ANALYSIS")
    print("="*60)
    
    # Samples per folder
    folder_counts = metadata.groupby('folder').size()
    print(f"\nTotal folders: {len(folder_counts)}")
    print(f"Mean samples per folder: {folder_counts.mean():.1f}")
    print(f"Median samples per folder: {folder_counts.median():.1f}")
    print(f"Min samples per folder: {folder_counts.min()}")
    print(f"Max samples per folder: {folder_counts.max()}")
    
    # Positives per folder (should be ~1)
    pos_per_folder = metadata[metadata['label'] == 1].groupby('folder').size()
    print(f"\nFolders with positives: {len(pos_per_folder)}")
    print(f"Mean positives per folder: {pos_per_folder.mean():.2f}")
    
    # Histogram
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(folder_counts, bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Samples per Folder')
    plt.ylabel('Count')
    plt.title('Distribution of Samples Across Folders')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    split_folder_counts = metadata.groupby(['split', 'folder']).size().unstack(fill_value=0)
    split_folder_counts.sum().plot(kind='bar', color='steelblue')
    plt.xlabel('Split')
    plt.ylabel('Total Folders')
    plt.title('Number of Folders per Split')
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(WORK_DIR / 'spatial_distribution.png', dpi=150)
    print(f"\n✓ Saved spatial distribution plot: {WORK_DIR / 'spatial_distribution.png'}")
    plt.close()


def analyze_channel_statistics(metadata, data_dir, n_samples=50):
    """Analyze per-channel statistics"""
    print("\n" + "="*60)
    print("PER-CHANNEL STATISTICS")
    print("="*60)
    
    # Sample stamps
    samples = metadata.sample(min(n_samples, len(metadata)))
    
    channel_stats = {f'ch{i}': {'mean': [], 'std': [], 'min': [], 'max': []} 
                     for i in range(5)}
    
    for _, row in tqdm(samples.iterrows(), total=len(samples), desc="Analyzing channels"):
        arr = np.load(data_dir / row['stamp_path'])['x']
        if arr.ndim == 3 and arr.shape[-1] == 5:
            arr = np.transpose(arr, (2, 0, 1))
        
        for i in range(5):
            channel_stats[f'ch{i}']['mean'].append(arr[i].mean())
            channel_stats[f'ch{i}']['std'].append(arr[i].std())
            channel_stats[f'ch{i}']['min'].append(arr[i].min())
            channel_stats[f'ch{i}']['max'].append(arr[i].max())
    
    # Print summary
    channel_names = ['F1 (Frame 1)', 'F2 (Frame 2)', 'F3 (Frame 3)', 
                     'S2 (F2-med)', 'S3 (F3-med)']
    
    for i, name in enumerate(channel_names):
        ch_key = f'ch{i}'
        print(f"\n{name}:")
        print(f"  Mean:  {np.mean(channel_stats[ch_key]['mean']):.3f} ± {np.std(channel_stats[ch_key]['mean']):.3f}")
        print(f"  Std:   {np.mean(channel_stats[ch_key]['std']):.3f} ± {np.std(channel_stats[ch_key]['std']):.3f}")
        print(f"  Range: [{np.mean(channel_stats[ch_key]['min']):.3f}, {np.mean(channel_stats[ch_key]['max']):.3f}]")
    
    # Box plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    for idx, stat_name in enumerate(['mean', 'std', 'min', 'max']):
        ax = axes[idx // 2, idx % 2]
        data = [channel_stats[f'ch{i}'][stat_name] for i in range(5)]
        ax.boxplot(data, labels=channel_names)
        ax.set_title(f'Channel {stat_name.capitalize()} Distribution')
        ax.set_ylabel(stat_name.capitalize())
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(WORK_DIR / 'channel_statistics.png', dpi=150)
    print(f"\n✓ Saved channel statistics plot: {WORK_DIR / 'channel_statistics.png'}")
    plt.close()


def main():
    print("="*60)
    print("DATASET DIAGNOSTICS")
    print("="*60)
    
    # Load metadata
    if not METADATA_CSV.exists():
        print(f"❌ Metadata not found at {METADATA_CSV}")
        return
    
    metadata = pd.read_csv(METADATA_CSV)
    print(f"✓ Loaded metadata: {len(metadata)} samples")
    
    WORK_DIR.mkdir(exist_ok=True, parents=True)
    
    # Run analyses
    analyze_class_balance(metadata)
    analyze_spatial_distribution(metadata)
    analyze_normalization(metadata, DATA_DIR, n_check=100)
    analyze_channel_statistics(metadata, DATA_DIR, n_samples=50)
    visualize_samples(metadata, DATA_DIR, n_samples=N_SAMPLES)
    
    print("\n" + "="*60)
    print("DIAGNOSTIC COMPLETE")
    print("="*60)
    print("\nGenerated files:")
    print(f"  - {WORK_DIR / 'class_balance.png'}")
    print(f"  - {WORK_DIR / 'spatial_distribution.png'}")
    print(f"  - {WORK_DIR / 'normalization_check.png'}")
    print(f"  - {WORK_DIR / 'channel_statistics.png'}")
    print(f"  - {WORK_DIR / 'samples_positive.png'}")
    print(f"  - {WORK_DIR / 'samples_negative.png'}")
    
    print("\n💡 Review these diagnostics before training!")
    print("   Key things to check:")
    print("   1. Normalization: mean~0, std~1-2, range [-5,5]")
    print("   2. No NaN/Inf values")
    print("   3. Class balance matches expectations (1:20)")
    print("   4. Sample stamps look correct (asteroid visible in F2/F3, difference in S2/S3)")


if __name__ == "__main__":
    main()

In [ ]:
meta.groupby(["split","label"]).size()

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path

DATA_DIR = Path("/kaggle/input/stamps-clean/stamps_clean")
meta = pd.read_csv(DATA_DIR / "metadata.csv")

bad_files = []
for i, row in meta.iterrows():
    path = DATA_DIR / row["stamp_path"]
    arr = np.load(path)["x"]
    if not np.isfinite(arr).all():  # checks NaN and Inf
        bad_files.append(path.name)

print(f"Found {len(bad_files)} bad files out of {len(meta)} total.")
print("Example bad files:", bad_files[:5])


In [ ]:
import numpy as np

nan_total = 0
nan_pos = 0
nan_neg = 0

for _, row in meta.iterrows():
    arr = np.load(DATA_DIR / row['stamp_path'])['x']
    if not np.isfinite(arr).all():
        nan_total += 1
        if row["label"] == 1:
            nan_pos += 1
        else:
            nan_neg += 1

print(f"Files with NaN: {nan_total}/{len(meta)}")
print(f"  ↳ Positives with NaN: {nan_pos}")
print(f"  ↳ Negatives with NaN: {nan_neg}")


In [ ]:
#!/usr/bin/env python3
"""
Improved 5-Channel Asteroid Detection Training
================================================
Research-backed improvements:
1. Fixed normalization (removed double normalization bug)
2. Focal Loss for extreme class imbalance (1:20)
3. Proper data augmentation (geometric only, no photometry changes)
4. Learning rate scheduling with warmup
5. Early stopping on recall @ high precision
6. Improved initialization for 5-channel input

Based on research from:
- ATLAS two-stage CNN (Rabeendran et al. 2021)
- ZTF DeepStreaks (Wang et al. 2022)
- NEA detection papers in project knowledge
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from pathlib import Path
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ===================== CONFIG =====================
DATA_DIR = Path("/kaggle/input/stamps-clean/stamps_clean")
WORK_DIR = Path("/kaggle/working")
ACTIVE_CSV = DATA_DIR / "metadata.csv"

# Training hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 50
BASE_LR = 1e-3
WEIGHT_DECAY = 1e-4
WARMUP_EPOCHS = 3

# Focal Loss parameters (research-recommended)
FOCAL_ALPHA = 0.25  # weight for positive class
FOCAL_GAMMA = 2.0   # focusing parameter (higher = more focus on hard examples)

# Early stopping (prioritize recall)
PATIENCE = 7
MIN_PRECISION = 0.80  # don't sacrifice too much precision

# Augmentation
AUG_PROB = 0.5

# Random seed
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# ===================== FOCAL LOSS =====================
class FocalLoss(nn.Module):
    """
    Focal Loss for addressing extreme class imbalance.
    
    From: Lin et al. "Focal Loss for Dense Object Detection" (2017)
    Used in: ATLAS CNN (Rabeendran 2021), multiple asteroid detection papers
    
    FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    
    where:
    - p_t is the model's estimated probability for the correct class
    - alpha_t balances positive/negative importance
    - gamma focuses on hard examples (gamma=0 -> standard CE)
    
    For 1:20 imbalance with high recall priority:
    - alpha=0.25 gives positive class 4x weight (compensates for 20:1 ratio partially)
    - gamma=2.0 down-weights easy negatives exponentially
    """
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        """
        Args:
            inputs: (N,) predicted logits
            targets: (N,) true labels (0 or 1)
        """
        # Convert to probabilities
        p = torch.sigmoid(inputs)
        
        # Compute focal loss components
        ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        p_t = p * targets + (1 - p) * (1 - targets)  # p if y=1, 1-p if y=0
        focal_weight = (1 - p_t) ** self.gamma
        
        # Alpha weighting (give more weight to positive class)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        
        focal_loss = alpha_t * focal_weight * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


# ===================== DATASET =====================
class AsteroidDataset(Dataset):
    """
    Dataset for 5-channel asteroid stamps.
    
    CRITICAL FIX: Data is already normalized during prep with MAD-based robust z-score.
    We do NOT re-normalize here (this was causing double normalization bug).
    
    Channels: [F1, F2, F3, S2, S3] where:
    - F1, F2, F3: temporal frames
    - S2 = F2 - median(F1, F3): difference image highlighting F2
    - S3 = F3 - median(F1, F2): difference image highlighting F3
    
    Each channel already normalized: (x - median) / (1.4826 * MAD), clipped to [-5, 5]
    """
    def __init__(self, csv_path, split, base_root, augment=False):
        meta = pd.read_csv(csv_path)
        self.df = meta[meta["split"] == split].reset_index(drop=True)
        self.base_root = Path(base_root)
                # In AsteroidDataset.__init__, after loading df:
        print(f"  Loading {split}: filtering NaN files...")
        valid_idx = []
        for idx, row in self.df.iterrows():
            arr = np.load(self.base_root / row["stamp_path"])["x"]
            if np.isfinite(arr).all():
                valid_idx.append(idx)
        self.df = self.df.iloc[valid_idx].reset_index(drop=True)
        self.augment = augment
        self.split = split

        
        
        # Track class distribution
        pos_count = (self.df["label"] == 1).sum()
        neg_count = (self.df["label"] == 0).sum()
        print(f"{split.upper()}: {pos_count} positives, {neg_count} negatives "
              f"(ratio 1:{neg_count/max(pos_count,1):.1f})")
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load preprocessed stamp (already normalized!)
        arr = np.load(self.base_root / row["stamp_path"])["x"].astype(np.float32)
        
        # Ensure channel-first format (C, H, W)
        if arr.ndim == 3 and arr.shape[-1] == 5:
            arr = np.transpose(arr, (2, 0, 1))
        
        # Handle any NaN/inf (should be rare after preprocessing)
        arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
        
        # GEOMETRIC AUGMENTATION ONLY (no photometry changes!)
        # Research shows: preserve photometric properties for asteroid detection
        if self.augment and self.split == "train":
            arr = self._augment(arr)
        
        x = torch.tensor(arr, dtype=torch.float32)
        y = torch.tensor([float(row["label"])], dtype=torch.float32)
        
        return x, y
    
    def _augment(self, arr):
        """
        Geometric augmentations only - NO brightness/contrast changes.
        
        From research: Photometric changes corrupt magnitude information.
        Safe augmentations: rotations, flips (preserve pixel values)
        """
        if np.random.rand() < AUG_PROB:
            # Random 90-degree rotation
            k = np.random.randint(0, 4)
            if k > 0:
                arr = np.rot90(arr, k=k, axes=(1, 2)).copy()
        
        if np.random.rand() < AUG_PROB:
            # Horizontal flip
            arr = np.flip(arr, axis=2).copy()
        
        if np.random.rand() < AUG_PROB:
            # Vertical flip  
            arr = np.flip(arr, axis=1).copy()
        
        return arr


# ===================== MODEL =====================
class EfficientNet5Channel(nn.Module):
    """
    EfficientNet-B0 adapted for 5-channel input.
    
    IMPROVED INITIALIZATION:
    Instead of just averaging RGB and duplicating, we:
    1. Use all 3 RGB channels for temporal channels (F1, F2, F3)
    2. Initialize difference channels (S2, S3) with Xavier/He initialization
    
    This gives the model better starting points for learning temporal vs difference features.
    """
    def __init__(self):
        super().__init__()
        
        # Load pretrained EfficientNet-B0
        try:
            base = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        except:
            base = models.efficientnet_b0(pretrained=True)
        
        # ===== IMPROVED 5-CHANNEL INITIALIZATION =====
        old_conv = base.features[0][0]
        new_conv = nn.Conv2d(
            in_channels=5,
            out_channels=old_conv.out_channels,
            kernel_size=old_conv.kernel_size,
            stride=old_conv.stride,
            padding=old_conv.padding,
            bias=False
        )
        
        with torch.no_grad():
            # Get pretrained RGB weights
            rgb_weights = old_conv.weight.data.clone()  # (out_ch, 3, k, k)
            
            # Initialize new 5-channel weights
            new_weights = torch.zeros(
                old_conv.out_channels, 5, 
                old_conv.kernel_size[0], old_conv.kernel_size[1]
            )
            
            # Channels 0,1,2 (F1, F2, F3): use pretrained RGB weights
            new_weights[:, 0:3, :, :] = rgb_weights
            
            # Channels 3,4 (S2, S3 - difference images): He initialization
            # These need to learn from scratch as they're difference images
            nn.init.kaiming_normal_(new_weights[:, 3:5, :, :], mode='fan_out', nonlinearity='relu')
            
            new_conv.weight.copy_(new_weights)
        
        base.features[0][0] = new_conv
        
        # Backbone and head
        self.backbone = base.features
        self.pool = nn.AdaptiveAvgPool2d(1)
        
        # Classification head with dropout (prevent overfitting on small positive class)
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1280, 256),
            nn.ReLU(),
            nn.Dropout(0.4),  # Research shows dropout helps with imbalanced data
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.head(x)
        return x


# ===================== LEARNING RATE SCHEDULE =====================
class WarmupCosineSchedule:
    """
    Learning rate schedule with warmup + cosine decay.
    
    From research: Warmup prevents instability with small positive class.
    Cosine decay smoothly reduces LR for better convergence.
    """
    def __init__(self, optimizer, warmup_epochs, total_epochs, base_lr, min_lr=1e-6):
        self.optimizer = optimizer
        self.warmup_epochs = warmup_epochs
        self.total_epochs = total_epochs
        self.base_lr = base_lr
        self.min_lr = min_lr
        self.current_epoch = 0
    
    def step(self):
        if self.current_epoch < self.warmup_epochs:
            # Linear warmup
            lr = self.base_lr * (self.current_epoch + 1) / self.warmup_epochs
        else:
            # Cosine decay
            progress = (self.current_epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)
            lr = self.min_lr + (self.base_lr - self.min_lr) * 0.5 * (1 + np.cos(np.pi * progress))
        
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        self.current_epoch += 1
        return lr


# ===================== METRICS =====================
def compute_metrics(y_true, y_pred, y_prob, threshold=0.5):
    """
    Compute comprehensive metrics for imbalanced binary classification.
    
    Focus on:
    - Recall (don't miss real asteroids)
    - Precision (keep FP rate manageable)
    - F1 score (balance)
    """
    y_pred_binary = (y_prob >= threshold).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'specificity': specificity,
        'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
    }


def find_optimal_threshold(y_true, y_prob, min_precision=0.80):
    """
    Find threshold that maximizes recall while maintaining minimum precision.
    
    Critical for asteroid detection: we want high recall but can't tolerate
    too many false positives (astronomer review time is expensive).
    """
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_prob)
    
    # Find thresholds meeting minimum precision
    valid_idx = precisions >= min_precision
    
    if not valid_idx.any():
        print(f"⚠️  Warning: No threshold achieves precision >= {min_precision}")
        # Fallback: maximize F1
        f1_scores = 2 * precisions * recalls / (precisions + recalls + 1e-10)
        best_idx = np.argmax(f1_scores[:-1])
        return thresholds[best_idx], precisions[best_idx], recalls[best_idx]
    
    # Among valid thresholds, pick one with highest recall
    valid_recalls = recalls[:-1][valid_idx[:-1]]
    valid_thresholds = thresholds[valid_idx[:-1]]
    valid_precisions = precisions[:-1][valid_idx[:-1]]
    
    best_idx = np.argmax(valid_recalls)
    
    return valid_thresholds[best_idx], valid_precisions[best_idx], valid_recalls[best_idx]


# ===================== TRAINING =====================
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_probs = []
    all_labels = []
    
    pbar = tqdm(loader, desc="Training")
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        logits = model(xb).squeeze(1)
        loss = criterion(logits, yb.squeeze(1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * xb.size(0)
        
        with torch.no_grad():
            probs = torch.sigmoid(logits).cpu().numpy()
            all_probs.extend(probs)
            all_labels.extend(yb.squeeze(1).cpu().numpy())
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(loader.dataset)
    metrics = compute_metrics(
        np.array(all_labels), 
        np.array(all_probs) >= 0.5,
        np.array(all_probs)
    )
    
    return avg_loss, metrics


@torch.no_grad()
def validate(model, loader, criterion, device, threshold=0.5):
    model.eval()
    total_loss = 0
    all_probs = []
    all_labels = []
    
    for xb, yb in tqdm(loader, desc="Validating"):
        xb, yb = xb.to(device), yb.to(device)
        
        logits = model(xb).squeeze(1)
        loss = criterion(logits, yb.squeeze(1))
        
        total_loss += loss.item() * xb.size(0)
        
        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.extend(probs)
        all_labels.extend(yb.squeeze(1).cpu().numpy())
    
    avg_loss = total_loss / len(loader.dataset)
    metrics = compute_metrics(
        np.array(all_labels),
        np.array(all_probs) >= threshold,
        np.array(all_probs),
        threshold=threshold
    )
    
    return avg_loss, metrics, np.array(all_probs), np.array(all_labels)


# ===================== MAIN =====================
def main():
    print("=" * 60)
    print("IMPROVED 5-CHANNEL ASTEROID DETECTION TRAINING")
    print("=" * 60)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nDevice: {device}")
    
    # ===== LOAD DATA =====
    print("\n" + "="*60)
    print("LOADING DATA")
    print("="*60)
    
    train_ds = AsteroidDataset(ACTIVE_CSV, "train", DATA_DIR, augment=True)
    val_ds = AsteroidDataset(ACTIVE_CSV, "val", DATA_DIR, augment=False)
    test_ds = AsteroidDataset(ACTIVE_CSV, "test", DATA_DIR, augment=False)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, 
                             num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                           num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=2, pin_memory=True)
    
    # ===== MODEL =====
    print("\n" + "="*60)
    print("INITIALIZING MODEL")
    print("="*60)
    
    model = EfficientNet5Channel().to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # ===== OPTIMIZER & LOSS =====
    criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    scheduler = WarmupCosineSchedule(optimizer, WARMUP_EPOCHS, NUM_EPOCHS, BASE_LR)
    
    print(f"\nLoss: Focal Loss (alpha={FOCAL_ALPHA}, gamma={FOCAL_GAMMA})")
    print(f"Optimizer: AdamW (lr={BASE_LR}, wd={WEIGHT_DECAY})")
    print(f"Scheduler: Warmup({WARMUP_EPOCHS}) + Cosine")
    
    # ===== TRAINING LOOP =====
    print("\n" + "="*60)
    print("TRAINING")
    print("="*60)
    
    best_recall = 0
    best_f1 = 0
    patience_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'val_recall': [], 'val_precision': [], 'val_f1': []}
    
    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print("-" * 60)
        
        # Learning rate step
        lr = scheduler.step()
        print(f"Learning rate: {lr:.6f}")
        
        # Train
        train_loss, train_metrics = train_epoch(model, train_loader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss:.4f} | "
              f"Prec: {train_metrics['precision']:.3f} | "
              f"Rec: {train_metrics['recall']:.3f} | "
              f"F1: {train_metrics['f1']:.3f}")
        
        # Validate
        val_loss, val_metrics, val_probs, val_labels = validate(
            model, val_loader, criterion, device
        )
        print(f"Val Loss:   {val_loss:.4f} | "
              f"Prec: {val_metrics['precision']:.3f} | "
              f"Rec: {val_metrics['recall']:.3f} | "
              f"F1: {val_metrics['f1']:.3f}")
        print(f"Val Confusion: TP={val_metrics['tp']}, FP={val_metrics['fp']}, "
              f"TN={val_metrics['tn']}, FN={val_metrics['fn']}")
        
        # Save history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_recall'].append(val_metrics['recall'])
        history['val_precision'].append(val_metrics['precision'])
        history['val_f1'].append(val_metrics['f1'])
        
        # Save best model (prioritize recall, then F1)
        if val_metrics['recall'] > best_recall or \
           (val_metrics['recall'] == best_recall and val_metrics['f1'] > best_f1):
            best_recall = val_metrics['recall']
            best_f1 = val_metrics['f1']
            torch.save(model.state_dict(), WORK_DIR / "best_model.pt")
            print(f"✓ Saved best model (Recall: {best_recall:.3f}, F1: {best_f1:.3f})")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"Patience: {patience_counter}/{PATIENCE}")
        
        # Early stopping
        if patience_counter >= PATIENCE:
            print(f"\n⚠️  Early stopping triggered after {epoch+1} epochs")
            break
    
    # ===== FINAL EVALUATION =====
    print("\n" + "="*60)
    print("FINAL EVALUATION ON TEST SET")
    print("="*60)
    
    # Load best model
    model.load_state_dict(torch.load(WORK_DIR / "best_model.pt"))
    
    # Get predictions
    _, test_metrics_default, test_probs, test_labels = validate(
        model, test_loader, criterion, device, threshold=0.5
    )
    
    print("\n--- Metrics with default threshold (0.5) ---")
    print(f"Precision: {test_metrics_default['precision']:.3f}")
    print(f"Recall:    {test_metrics_default['recall']:.3f}")
    print(f"F1 Score:  {test_metrics_default['f1']:.3f}")
    print(f"Confusion Matrix:")
    print(f"  TP={test_metrics_default['tp']}, FP={test_metrics_default['fp']}")
    print(f"  FN={test_metrics_default['fn']}, TN={test_metrics_default['tn']}")
    
    # Find optimal threshold
    opt_thresh, opt_prec, opt_rec = find_optimal_threshold(
        test_labels, test_probs, min_precision=MIN_PRECISION
    )
    
    print(f"\n--- Optimal threshold (min precision={MIN_PRECISION}) ---")
    print(f"Threshold: {opt_thresh:.4f}")
    print(f"Precision: {opt_prec:.3f}")
    print(f"Recall:    {opt_rec:.3f}")
    
    test_metrics_opt = compute_metrics(
        test_labels, test_probs >= opt_thresh, test_probs, threshold=opt_thresh
    )
    print(f"F1 Score:  {test_metrics_opt['f1']:.3f}")
    print(f"Confusion Matrix:")
    print(f"  TP={test_metrics_opt['tp']}, FP={test_metrics_opt['fp']}")
    print(f"  FN={test_metrics_opt['fn']}, TN={test_metrics_opt['tn']}")
    
    # Save results
    results = {
        'test_metrics_default': test_metrics_default,
        'test_metrics_optimal': test_metrics_opt,
        'optimal_threshold': opt_thresh,
        'history': history
    }
    
        # Save full results (including predictions for later visualization)
    results = {
        'test_metrics_default': test_metrics_default,
        'test_metrics_optimal': test_metrics_opt,
        'optimal_threshold': opt_thresh,
        'history': history,
        'test_labels': test_labels,
        'test_probs': test_probs
    }

    torch.save(results, WORK_DIR / "training_results.pt")
    print(f"✓ Saved results with predictions: {WORK_DIR / 'training_results.pt'}")

    
    print("\n" + "="*60)
    print("TRAINING COMPLETE")
    print("="*60)
    print(f"Best model saved: {WORK_DIR / 'best_model.pt'}")
    print(f"Results saved: {WORK_DIR / 'training_results.pt'}")


if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
"""
Visualize TP / FP / FN / TN samples after model training
=========================================================
Loads saved test predictions and metadata,
then shows 1×5 channel strips for each classification category.
"""

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from pathlib import Path

WORK_DIR = Path("/kaggle/working")
DATA_DIR = Path("/kaggle/input/stamps-clean/stamps_clean")
META_PATH = DATA_DIR / "metadata.csv"

def visualize_classification_examples(metadata, y_true, y_prob, threshold, n_samples=6):
    """
    Visualize TP, FP, FN, TN examples as 1×5 channel strips.
    Each row shows [F₁–S₃] with per-row probability label (p̂).
    Layout is compact with minimal gap between label and image.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    y_pred = (y_prob >= threshold).astype(int)

    # Indices for each outcome type
    tp_idx = np.where((y_true == 1) & (y_pred == 1))[0]
    fp_idx = np.where((y_true == 0) & (y_pred == 1))[0]
    fn_idx = np.where((y_true == 1) & (y_pred == 0))[0]
    tn_idx = np.where((y_true == 0) & (y_pred == 0))[0]

    print(f"TP={len(tp_idx)}, FP={len(fp_idx)}, FN={len(fn_idx)}, TN={len(tn_idx)}")

    channel_labels = ["F₁", "F₂", "F₃", "S₂", "S₃"]

    def plot_subset(indices, title):
        if len(indices) == 0:
            print(f"No {title} examples found.")
            return

        idxs = np.random.choice(indices, min(n_samples, len(indices)), replace=False)
        fig, axes = plt.subplots(len(idxs), 5, figsize=(12, 2.2 * len(idxs)))
        fig.suptitle(title, fontsize=13, y=0.99)

        if len(idxs) == 1:
            axes = np.expand_dims(axes, axis=0)

        for i, idx in enumerate(idxs):
            row = metadata.iloc[idx]
            arr = np.load(DATA_DIR / row["stamp_path"])["x"]

            # Draw each channel with top labels
            for j in range(5):
                ax = axes[i, j]
                ax.imshow(arr[j], cmap="gray", vmin=-3, vmax=3)
                ax.set_title(channel_labels[j], fontsize=9, pad=2)
                ax.axis("off")

            # Probability label (closer to image)
            prob_text = f"p̂ = {y_prob[idx]:.3f}"
            axes[i, 0].text(
                -0.25, 0.5, prob_text,   # 👈 tighter spacing (was -0.6)
                va="center", ha="right",
                fontsize=9, color="black",
                transform=axes[i, 0].transAxes
            )

        plt.subplots_adjust(
            left=0.05, right=0.98, top=0.94, bottom=0.05,
            wspace=0.05, hspace=0.15
        )
        plt.show()

    # Plot each outcome group
    plot_subset(tp_idx, "True Positives (Correct Asteroids)")
    plot_subset(fp_idx, "False Positives (Non-Asteroids Misclassified)")
    plot_subset(fn_idx, "False Negatives (Missed Asteroids)")
    plot_subset(tn_idx, "True Negatives (Correct Non-Asteroids)")

def main():
    print("="*60)
    print("VISUALIZING TEST CLASSIFICATIONS")
    print("="*60)

    # Load results with predictions
    results_path = WORK_DIR / "training_results.pt"
    results = torch.load(results_path, weights_only=False)
    y_true = np.array(results["test_labels"])
    y_prob = np.array(results["test_probs"])
    threshold = results["optimal_threshold"]

    # Load metadata
    metadata = pd.read_csv(META_PATH)
    metadata = metadata[metadata["split"] == "test"].reset_index(drop=True)

    visualize_classification_examples(metadata, y_true, y_prob, threshold)

if __name__ == "__main__":
    main()


In [ ]:
def visualize_classification_examples(metadata, y_true, y_prob, optimal_thresh, n_samples=6):
    """
    Visualize TP, FP, FN, TN examples from test dataset.
    Each example shows a 1×5 strip of the five channels [F1, F2, F3, S2, S3].
    """
    import random

    y_pred = (y_prob >= optimal_thresh).astype(int)

    # Identify indices
    tp_idx = np.where((y_true == 1) & (y_pred == 1))[0]
    fp_idx = np.where((y_true == 0) & (y_pred == 1))[0]
    fn_idx = np.where((y_true == 1) & (y_pred == 0))[0]
    tn_idx = np.where((y_true == 0) & (y_pred == 0))[0]

    print(f"\nTP={len(tp_idx)}, FP={len(fp_idx)}, FN={len(fn_idx)}, TN={len(tn_idx)}")

    def show_examples(idxs, title):
        if len(idxs) == 0:
            print(f"No {title.lower()} examples available.")
            return
        subset = random.sample(list(idxs), min(n_samples, len(idxs)))
        fig, axes = plt.subplots(len(subset), 5, figsize=(15, 3*len(subset)))
        fig.suptitle(title, fontsize=14)
        for i, idx in enumerate(subset):
            row = metadata.iloc[idx]
            path = DATA_DIR / row['stamp_path']
            arr = np.load(path)['x']  # shape (5, H, W)
            for j in range(5):
                ax = axes[i, j] if len(subset) > 1 else axes[j]
                ax.imshow(arr[j], cmap='gray', vmin=-3, vmax=3)
                if i == 0:
                    ax.set_title(['F1','F2','F3','S2','S3'][j])
                ax.axis('off')
        plt.tight_layout()
        plt.show()

    # Show examples per class
    show_examples(tp_idx, "True Positives (Correct Asteroid Detections)")
    show_examples(fp_idx, "False Positives (Non-Asteroids Classified as Asteroids)")
    show_examples(fn_idx, "False Negatives (Missed Asteroids)")
    show_examples(tn_idx, "True Negatives (Correct Non-Asteroids)")



In [ ]:
    # === Visualize TP, FP, FN, TN samples ===
    print("\n" + "="*60)
    print("VISUALIZING SAMPLE CLASSIFICATIONS")
    print("="*60)

    # Load predictions if saved, else skip
    if "test_labels" in results and "test_probs" in results:
        y_true = np.array(results["test_labels"])
        y_prob = np.array(results["test_probs"])
        visualize_classification_examples(metadata, y_true, y_prob, optimal_thresh)
    else:
        print("⚠️ Skipping visualization: test predictions not saved in results.")
